# Cryptography project -  Maëlys Bühler

This notebooks only works if OpenSSL is installed on the system it's ran on. If an error with OpenSLL occurs, please make sure that OpenSSL is installed.
The notebook has been tested on Ubuntu 20.04.6 LTS, running in WSL1.

This codes prepares the environnement and create some functions used in the next steps. It must be ran before running any other cell

In [18]:
import subprocess
import os
import shutil

def call_command(command):
    print("$ " + command)
    try:
        subproc = subprocess.run(command.split())    
        if(subproc.returncode != 0):
            print("Error:")
            print("Return code: " + str(subproc.returncode))
            exit(1)    
    except Exception as e:
        print("Error: " + e.__class__.__name__ )
        print(str(e))
        exit(1)

def remove_files(files):
    for file in files:
        if os.path.exists(file):
            os.remove(file)

def remove_directory(dir):
    if os.path.exists(dir):
        shutil.rmtree(dir)

def create_directory(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)


directories = ["RSA", "sym_keys", "messages", "messages/plain", "messages/encrypted", "messages/decrypted", "messages/hashed", "messages/signed", "DSA"]
#create directorys
for dir in directories:
    remove_directory(dir)
    create_directory(dir)

## 1. Using assymetric cypher for key exchange: RSA
Alice is going to generate a key pair and send the public key to Bob.  
To do this, she's going to begin by generating her private key.
Using openssl, the command is the following:
```
openssl genpkey -algorithm RSA -out ./RSA/alice_private_key.pem -pkeyopt rsa_keygen_bits:1024
```
She's then going to generate the public key linked to this private key. To do so, she uses the following command:
```
openssl rsa -pubout -in ./RSA/alice_private_key.pem -out ./RSA/alice_public_key.pem
```

In [19]:
#if files already exist, remove them
remove_files(["alice_private_key.pem", "alice_public_key.pem"])

create_private_key = "openssl genpkey -algorithm RSA -out ./RSA/alice_private_key.pem -pkeyopt rsa_keygen_bits:1024"
create_public_key = "openssl rsa -pubout -in ./RSA/alice_private_key.pem -out ./RSA/alice_public_key.pem"
call_command(create_private_key)
call_command(create_public_key)

$ openssl genpkey -algorithm RSA -out ./RSA/alice_private_key.pem -pkeyopt rsa_keygen_bits:1024
$ openssl rsa -pubout -in ./RSA/alice_private_key.pem -out ./RSA/alice_public_key.pem


......................+++++
.............................................................................................+++++
writing RSA key


Now that Alice has her public key, she sends it to Bob, or simply publish it for everyone to find it.  
Bob wants to have a secret conversation with her, using a symmetric cypher. He begins by generating a key:
```
openssl rand -base64 -out ./sym_keys/alice_bob_sym_key.plain 32
```
He then needs to send to Alice the symmetric key, but to make sure that no-one else can read this symmetric key, he encrypt it using Alice's public key.  
  
He first put the symmetric key in a file. Let's call it "sym_key.plain"
Then, he encrypt the file with RSA, using Alice's public key with the following command:
```
openssl rsautl -encrypt -inkey ./RSA/alice_public_key.pem -pubin -in ./sym_keys/alice_bob_sym_key.plain -out ./sym_keys/alice_bob_sym_key.rsa
```
He can then send the encrypted file, sym_key.rsa, to Alice. This file can only be decrypted with Alice's private key, and Alice is the only one who has it.
Alice, when she received the file, decrypt it:
```
openssl rsautl -decrypt -inkey ./RSA/alice_private_key.pem -in ./sym_keys/alice_bob_sym_key.rsa -out ./sym_keys/alice_bob_sym_key.dec
```
She now also has the symmetric key, in the file sym_key.dec

In [20]:
#if files already exist, remove them
remove_files(["sym_key.plain", "sym_key.rsa", "sym_key.dec"])

#create symmetric key
create_symmetric_key = "openssl rand -base64 -out ./sym_keys/alice_bob_sym_key.plain 32"
display_symmetric_key = "cat ./sym_keys/alice_bob_sym_key.plain"
#encrypt symmetric key
encrypt_symmetric_key = "openssl rsautl -encrypt -inkey ./RSA/alice_public_key.pem -pubin -in ./sym_keys/alice_bob_sym_key.plain -out ./sym_keys/alice_bob_sym_key.rsa"
display_encrypted_symmetric_key = "cat ./sym_keys/alice_bob_sym_key.rsa"
#decrypt symmetric key
decrypt_symmetric_key = "openssl rsautl -decrypt -inkey ./RSA/alice_private_key.pem -in ./sym_keys/alice_bob_sym_key.rsa -out ./sym_keys/alice_bob_sym_key.dec"
display_decrypted_symmetric_key = "cat ./sym_keys/alice_bob_sym_key.dec"
call_command(create_symmetric_key)
call_command(display_symmetric_key)
call_command(encrypt_symmetric_key)
call_command(display_encrypted_symmetric_key)
print()
call_command(decrypt_symmetric_key)
call_command(display_decrypted_symmetric_key)

$ openssl rand -base64 -out ./sym_keys/alice_bob_sym_key.plain 32
$ cat ./sym_keys/alice_bob_sym_key.plain
/IcckmPr86b1ic9FdDw5sMe1ZKcaILyzubKh4yN1fBQ=
$ openssl rsautl -encrypt -inkey ./RSA/alice_public_key.pem -pubin -in ./sym_keys/alice_bob_sym_key.plain -out ./sym_keys/alice_bob_sym_key.rsa
$ cat ./sym_keys/alice_bob_sym_key.rsa
��Kx��JmvP瞍:��b���ޤz5���Ԛ�j��G�tO�e��oN̔zW	��Ʊ ߨ��ΩP���f�t�`�P߶:��� M��.�w3��dO���EA��غ�M��)j�π2�R�
$ openssl rsautl -decrypt -inkey ./RSA/alice_private_key.pem -in ./sym_keys/alice_bob_sym_key.rsa -out ./sym_keys/alice_bob_sym_key.dec
$ cat ./sym_keys/alice_bob_sym_key.dec


/IcckmPr86b1ic9FdDw5sMe1ZKcaILyzubKh4yN1fBQ=


## 2. Using symmetric cipher to encrypt/decrypt:
### 2.1 RC4 Stream Cipher
Now that both Bob and Alice have the symmetric key for encrypting and decrypting the messages, Bob wants to send a message to Alice.  

To do this, he write a message in a file named "message.txt".  

To encrypt the message, Bob uses the following command:
```
openssl enc -rc4 -a -pbkdf2 -iter 1000000 -salt -in ./messages/plain/message1_from_bob.txt -out ./messages/encrypted/message1_from_bob.b64 -k <symmetric_key>
```
Bob can then send the message to Alice.  
Alice can decrypt it using:
```
openssl enc -rc4 -d -a -pbkdf2 -iter 1000000 -in ./messages/encrypted/message1_from_bob.b64 -out ./messages/decrypted/message1_from_bob.dec -k <symmetric_key>
```

In [21]:
#if files already exist, remove them
remove_files(["./messages/plain/message1_from_bob.txt", "./messages/encrypted/message1_from_bob.b64", "./messages/decrypted/message1_from_bob.dec"])

key = "symmetric_key"
with open("./sym_keys/alice_bob_sym_key.dec", "r") as f:
    key = f.read()
print("Key: " + key)

with open("./messages/plain/message1_from_bob.txt", "x") as f:
    f.write("This is a test message.")

display_plain_message = "cat ./messages/plain/message1_from_bob.txt"
encrypt_message = f"openssl enc -rc4 -a -pbkdf2 -iter 1000000 -salt -in ./messages/plain/message1_from_bob.txt -out ./messages/encrypted/message1_from_bob.b64 -k {key}"
display_encrypted_message = "cat ./messages/encrypted/message1_from_bob.b64"
decypt_message = f"openssl enc -rc4 -d -a -pbkdf2 -iter 1000000 -in ./messages/encrypted/message1_from_bob.b64 -out ./messages/decrypted/message1_from_bob.dec -k {key}"
display_decrypted_message = "cat ./messages/decrypted/message1_from_bob.dec"

call_command(display_plain_message)
print("\n")
call_command(encrypt_message)
call_command(display_encrypted_message)
print()
call_command(decypt_message)
call_command(display_decrypted_message)

Key: /IcckmPr86b1ic9FdDw5sMe1ZKcaILyzubKh4yN1fBQ=

$ cat ./messages/plain/message1_from_bob.txt
This is a test message.

$ openssl enc -rc4 -a -pbkdf2 -iter 1000000 -salt -in ./messages/plain/message1_from_bob.txt -out ./messages/encrypted/message1_from_bob.b64 -k /IcckmPr86b1ic9FdDw5sMe1ZKcaILyzubKh4yN1fBQ=

$ cat ./messages/encrypted/message1_from_bob.b64
U2FsdGVkX1/f4jyhF4RkVQW+wWAqJgL4wfAgXHHOQoeCVUVfc9aU

$ openssl enc -rc4 -d -a -pbkdf2 -iter 1000000 -in ./messages/encrypted/message1_from_bob.b64 -out ./messages/decrypted/message1_from_bob.dec -k /IcckmPr86b1ic9FdDw5sMe1ZKcaILyzubKh4yN1fBQ=

$ cat ./messages/decrypted/message1_from_bob.dec
This is a test message.

### 2.2 EAS block cipher in CFB mode
RC4 is a stream cipher. This means that every digit is encrypted one at the time. It is therefore not usable with block cipher mode like CFB.  
To demonstrate the use of CFB, we'll use the aes256 algorithm, which is a block cypher.

```
openssl enc -aes-256-cfb -a -pbkdf2 -iter 1000000 -salt -in ./messages/plain/message1_from_bob.txt -out ./messages/encrypted/message1_from_bob.b64 -k <symmetric_key>
```
Bob can then send the message to Alice.  
Alice can decrypt it using:
```
openssl enc -aes-256-cfb -d -a -pbkdf2 -iter 1000000 -in ./messages/encrypted/message1_from_bob.b64 -out ./messages/decrypted/message1_from_bob.dec -k <symmetric_key>
```

In [22]:
#if files already exist, remove them
remove_files(["./messages/plain/message1_from_bob.txt", "./messages/encrypted/message1_from_bob.b64", "./messages/decrypted/message1_from_bob.dec"])

key = "symmetric_key"
with open("./sym_keys/alice_bob_sym_key.dec", "r") as f:
    key = f.read()
print("Key: " + key)

with open("./messages/plain/message1_from_bob.txt", "x") as f:
    f.write("This is a test message.")

display_plain_message = "cat ./messages/plain/message1_from_bob.txt"
encrypt_message = f"openssl enc -aes-256-cfb -a -pbkdf2 -iter 1000000 -salt -in ./messages/plain/message1_from_bob.txt -out ./messages/encrypted/message1_from_bob.b64 -k {key}"
display_encrypted_message = "cat ./messages/encrypted/message1_from_bob.b64"
decypt_message = f"openssl enc -aes-256-cfb -d -a -pbkdf2 -iter 1000000 -in ./messages/encrypted/message1_from_bob.b64 -out ./messages/decrypted/message1_from_bob.dec -k {key}"
display_decrypted_message = "cat ./messages/decrypted/message1_from_bob.dec"

call_command(display_plain_message)
print("\n")
call_command(encrypt_message)
call_command(display_encrypted_message)
print()
call_command(decypt_message)
call_command(display_decrypted_message)

Key: /IcckmPr86b1ic9FdDw5sMe1ZKcaILyzubKh4yN1fBQ=

$ cat ./messages/plain/message1_from_bob.txt
This is a test message.

$ openssl enc -aes-256-cfb -a -pbkdf2 -iter 1000000 -salt -in ./messages/plain/message1_from_bob.txt -out ./messages/encrypted/message1_from_bob.b64 -k /IcckmPr86b1ic9FdDw5sMe1ZKcaILyzubKh4yN1fBQ=

$ cat ./messages/encrypted/message1_from_bob.b64
U2FsdGVkX1+lQW46c2I9nMGY/Pdn9K83GB1HwB8mD7eiJfp0TaJ1

$ openssl enc -aes-256-cfb -d -a -pbkdf2 -iter 1000000 -in ./messages/encrypted/message1_from_bob.b64 -out ./messages/decrypted/message1_from_bob.dec -k /IcckmPr86b1ic9FdDw5sMe1ZKcaILyzubKh4yN1fBQ=

$ cat ./messages/decrypted/message1_from_bob.dec
This is a test message.

## 3. Use a cryptographic hash: SHA224

Alice wants to hash a message, with the SHA224 cryptographic hash.
To do so, she uses the command:
```
openssl dgst -sha224 -out ./messages/hashed/message2_from_alice.hash ./messages/plain/message2_from_alice.txt
```
She can then find the hash in the hash.txt file.

In [23]:
#if files already exist, remove them
remove_files(["./messages/hashed/message2_from_alice.hash", "./messages/plain/message2_from_alice.txt"])

with open("./messages/plain/message2_from_alice.txt", "x") as f:
    f.write("This is a test message from Alice.")

hash_message = "openssl dgst -sha224 -out ./messages/hashed/message2_from_alice.hash ./messages/plain/message2_from_alice.txt"
display_hash = "cat ./messages/hashed/message2_from_alice.hash"
call_command(hash_message)
call_command(display_hash)


$ openssl dgst -sha224 -out ./messages/hashed/message2_from_alice.hash ./messages/plain/message2_from_alice.txt
$ cat ./messages/hashed/message2_from_alice.hash
SHA224(./messages/plain/message2_from_alice.txt)= 6dbee1454fade32c4e4ba7a950bac91ad5e2513d95622989572380a2


## 4. Sign a message: DSA

Alice wants to use the hash of the message to sign it with DSA.  
For that she'll need DSA's public and private keys.
She'll begin by generating DSA parameters used for the keys generation:
```
openssl dsaparams -out ./DSA/dsa_param.pem 1024
```
Using these parameters, she generate her private key
```
openssl gendsa -out ./DSA/alice_private_key.pem dsa_param.pem
```
And her public key:
```
openssl dsa -in ./DSA/alice_private_key.pem -pubout -out ./DSA/alice_public_key.pem
```

In [24]:
#if files already exist, remove them
remove_files(["./DSA/dsa_param.pem", "./DSA/alice_private_key.pem", "./DSA/alice_public_key.pem"])

gen_param = "openssl dsaparam -out ./DSA/dsa_param.pem 1024"
gen_private_key = "openssl gendsa -out ./DSA/alice_private_key.pem ./DSA/dsa_param.pem"
gen_public_key = "openssl dsa -in ./DSA/alice_private_key.pem -pubout -out ./DSA/alice_public_key.pem"

call_command(gen_param)
call_command(gen_private_key)
call_command(gen_public_key)

$ openssl dsaparam -out ./DSA/dsa_param.pem 1024
$ openssl gendsa -out ./DSA/alice_private_key.pem ./DSA/dsa_param.pem
$ openssl dsa -in ./DSA/alice_private_key.pem -pubout -out ./DSA/alice_public_key.pem


Generating DSA parameters, 1024 bit long prime
This could take some time
....+.........+.........+.+...+...........+.......+.........+.......+.+......+.........................+......................................................+...+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++*
+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++*
Generating DSA key, 1024 bits
read DSA key
writing DSA key


She uses her DSA private key to sign the hash, using the following command:
```
openssl dgst -sha224 -sign ./DSA/alice_private_key.pem -out ./messages/signed/message3_from_alice.sign ./messages/hashed/message3_from_alice.hash
```
She then sent the message and the signature to Bob.
Bob, after receiving the message, wants to verify the signature. To do so, he get the sha224 hash of the message in the same Alice did before.
```
openssl dgst -sha224 -out ./messages/hashed/message3_from_alice.hash ./messages/plain/message3_from_alice.txt
```
He can now compare the signature and the hash of the message using:
```
openssl dgst -sha224 -verify ./DSA/alice_public_key.pem -signature ./messages/signed/message3_from_alice.sign ./messages/hashed/message3_from_alice.hash
```

In [25]:
#if files already exist, remove them
remove_files(["./messages/signed/message3_from_alice.sign", ])

hash_of_alice_message = "openssl dgst -sha224 -out ./messages/hashed/message3_from_alice.hash ./messages/plain/message3_from_alice.txt"
signature_message = "openssl dgst -sha224 -sign ./DSA/alice_private_key.pem -out ./messages/signed/message3_from_alice.sign ./messages/hashed/message3_from_alice.hash"
display_signature = "cat ./messages/signed/message3_from_alice.sign"
display_hash_of_alice_message = "cat ./messages/hashed/message3_from_alice.hash"
verify_signature = "openssl dgst -sha224 -verify ./DSA/alice_public_key.pem -signature ./messages/signed/message3_from_alice.sign ./messages/hashed/message3_from_alice.hash"
call_command(hash_of_alice_message)
call_command(signature_message)
call_command(display_signature)
print()
#bob do not have alice's message hash, he needs to hash the message first
remove_files(["./messages/hashed/message3_from_alice.hash", ])
call_command(hash_of_alice_message)
call_command(display_hash_of_alice_message)
call_command(verify_signature)


$ openssl dgst -sha224 -out ./messages/hashed/message3_from_alice.hash ./messages/plain/message3_from_alice.txt


Error:
Return code: 1
$ openssl dgst -sha224 -sign ./DSA/alice_private_key.pem -out ./messages/signed/message3_from_alice.sign ./messages/hashed/message3_from_alice.hash
$ cat ./messages/signed/message3_from_alice.sign
0,O��2b���rR���=�ԉ�3Z��'q�CcXs޶���[
$ openssl dgst -sha224 -out ./messages/hashed/message3_from_alice.hash ./messages/plain/message3_from_alice.txt


./messages/plain/message3_from_alice.txt: No such file or directory
./messages/plain/message3_from_alice.txt: No such file or directory


Error:
Return code: 1
$ cat ./messages/hashed/message3_from_alice.hash
$ openssl dgst -sha224 -verify ./DSA/alice_public_key.pem -signature ./messages/signed/message3_from_alice.sign ./messages/hashed/message3_from_alice.hash
Verified OK


## 5. Sign then encrypt VS encrypt then sign
### Sign, then encrypt
Alice wants to send an encrypted and signed message to Bob.
She decides to sign it before encrypting it.

To do so, she gets the hash of his message, then use it to get the signature, and then she encrypt the message, using the key exchanged in chapter 1.
```
openssl dgst -sha224 -out ./messages/hashed/message3_from_alice.hash ./messages/plain/message3_from_alice.txt 
openssl dgst -sha224 -sign ./RSA_keys/alice_private_key.pem -out ./messages/signed/message3_from_alice.sign ./messages/hashed/message3_from_alice.hash
openssl enc -rc4 -a -pbkdf2 -iter 1000000 -salt -in ./messages/plain/message3_from_alice.txt -out ./messages/encrypted/message3_from_alice.b64 -k <symmetric_key>
```

In [26]:
#if files already exist, remove them
remove_files(["./messages/plain/message3_from_alice.txt", "./messages/signed/message3_from_alice.sign", "./messages/hashed/message3_from_alice.hash", "./messages/encrypted/message3_from_alice.b64"])

with open("./messages/plain/message3_from_alice.txt", "x") as f:
    f.write("This is a test message from Alice.")


hash_message = "openssl dgst -sha224 -out ./messages/hashed/message3_from_alice.hash ./messages/plain/message3_from_alice.txt"
display_hash = "cat ./messages/hashed/message3_from_alice.hash"
sign_message = "openssl dgst -sha224 -sign ./RSA/alice_private_key.pem -out ./messages/signed/message3_from_alice.sign ./messages/hashed/message3_from_alice.hash"
display_signature = "cat ./messages/signed/message3_from_alice.sign"
encrypt_message = f"openssl enc -rc4 -a -pbkdf2 -iter 1000000 -salt -in ./messages/plain/message3_from_alice.txt -out ./messages/encrypted/message3_from_alice.b64 -k {key}"
display_encrypted_message = "cat ./messages/encrypted/message3_from_alice.b64"

call_command(hash_message)
call_command(display_hash)
call_command(sign_message)
call_command(display_signature)
print()
call_command(encrypt_message)
call_command(display_encrypted_message)

$ openssl dgst -sha224 -out ./messages/hashed/message3_from_alice.hash ./messages/plain/message3_from_alice.txt


$ cat ./messages/hashed/message3_from_alice.hash
SHA224(./messages/plain/message3_from_alice.txt)= 6dbee1454fade32c4e4ba7a950bac91ad5e2513d95622989572380a2
$ openssl dgst -sha224 -sign ./RSA/alice_private_key.pem -out ./messages/signed/message3_from_alice.sign ./messages/hashed/message3_from_alice.hash
$ cat ./messages/signed/message3_from_alice.sign
-<�\�E��&���.��~�:j�����n{�͓�G"W�����r���n��4��5���`V!�Ԉb�W�qW�~V�ت:�o�[@��2��5������ˌU1U��+���IŸ?��ci����E&
$ openssl enc -rc4 -a -pbkdf2 -iter 1000000 -salt -in ./messages/plain/message3_from_alice.txt -out ./messages/encrypted/message3_from_alice.b64 -k /IcckmPr86b1ic9FdDw5sMe1ZKcaILyzubKh4yN1fBQ=

$ cat ./messages/encrypted/message3_from_alice.b64
U2FsdGVkX1+2VAB5OfLWaDP1ZOVttL3uszOF95go+fY24qLjvtVGSqiKR5ipfwTN
TEw=


Bob is now going to decrypt the file, then check the signature by getting the hash of the decrypted message, and comparing it with the signature.
```
openssl enc -rc4 -d -a -pbkdf2 -iter 1000000 -in ./messages/encrypted/message3_from_alice.b64 -out ./messages/plain/message3_from_alice.txt -k <symmetric_key>
openssl dgst -sha224 -out ./messages/hashed/message3_from_alice.hash ./messages/plain/message3_from_alice.txt
openssl dgst -sha224 -verify ./RSA_keys/alice_public_key.pem -signature ./messages/signed/message3_from_alice.sign ./messages/hashed/message3_from_alice.hash
```


In [27]:
#if files already exist, remove them
remove_files(["./messages/decrypted/message3_from_alice.dec", "./messages/hashed/message3_from_alice.hash", "./messages/plain/message3_from_alice.txt"])

decrypt_message = f"openssl enc -rc4 -d -a -pbkdf2 -iter 1000000 -in ./messages/encrypted/message3_from_alice.b64 -out ./messages/plain/message3_from_alice.txt -k {key}"
display_decrypted_message = "cat ./messages/plain/message3_from_alice.txt"
hash_of_alice_message = "openssl dgst -sha224 -out ./messages/hashed/message3_from_alice.hash ./messages/plain/message3_from_alice.txt"
display_hash_of_alice_message = "cat ./messages/hashed/message3_from_alice.hash"
verify_signature = "openssl dgst -sha224 -verify ./RSA/alice_public_key.pem -signature ./messages/signed/message3_from_alice.sign ./messages/hashed/message3_from_alice.hash"

call_command(decrypt_message)
call_command(display_decrypted_message)
print()
call_command(hash_of_alice_message)
call_command(display_hash_of_alice_message)
call_command(verify_signature)

$ openssl enc -rc4 -d -a -pbkdf2 -iter 1000000 -in ./messages/encrypted/message3_from_alice.b64 -out ./messages/plain/message3_from_alice.txt -k /IcckmPr86b1ic9FdDw5sMe1ZKcaILyzubKh4yN1fBQ=

$ cat ./messages/plain/message3_from_alice.txt
This is a test message from Alice.
$ openssl dgst -sha224 -out ./messages/hashed/message3_from_alice.hash ./messages/plain/message3_from_alice.txt
$ cat ./messages/hashed/message3_from_alice.hash
SHA224(./messages/plain/message3_from_alice.txt)= 6dbee1454fade32c4e4ba7a950bac91ad5e2513d95622989572380a2
$ openssl dgst -sha224 -verify ./RSA/alice_public_key.pem -signature ./messages/signed/message3_from_alice.sign ./messages/hashed/message3_from_alice.hash
Verified OK


### Encrypt, then sign
Alice wants to send an encrypted and signed message to Bob.
This time, she decides to encrypt it before signing it.

To do so, she encrypt the message, then get the hash of the encrypted message and use it to sign the message.
```
openssl enc -rc4 -a -pbkdf2 -iter 1000000 -salt -in ./messages/plain/message4_from_alice.txt -out ./messages/encrypted/message4_from_alice.b64 -k<symmetric_key>
openssl dgst -sha224 -out ./messages/hashed/message4_from_alice.hash ./messages/encrypted/message4_from_alice.b64
openssl dgst -sha224 -sign ./DSA/alice_private_key.pem -out ./messages/signed/message4_from_alice.sign ./messages/hashed/message4_from_alice.hash
```

In [28]:
#if files already exist, remove them
remove_files(["./messages/plain/message4_from_alice.txt", "./messages/encrypted/message4_from_alice.b64", "./messages/hashed/message4_from_alice.hash", "./messages/signed/message4_from_alice.sign"])

with open("./messages/plain/message4_from_alice.txt", "x") as f:
    f.write("This is a test message from Alice.")

encrypt_message = f"openssl enc -rc4 -a -pbkdf2 -iter 1000000 -salt -in ./messages/plain/message4_from_alice.txt -out ./messages/encrypted/message4_from_alice.b64 -k {key}"
display_encrypted_message = "cat ./messages/encrypted/message4_from_alice.b64"
hash_message = "openssl dgst -sha224 -out ./messages/hashed/message4_from_alice.hash ./messages/encrypted/message4_from_alice.b64"
display_hash = "cat ./messages/hashed/message4_from_alice.hash"
sign_message = "openssl dgst -sha224 -sign ./DSA/alice_private_key.pem -out ./messages/signed/message4_from_alice.sign ./messages/hashed/message4_from_alice.hash"
display_signature = "cat ./messages/signed/message4_from_alice.sign"

call_command(encrypt_message)
call_command(display_encrypted_message)
call_command(hash_message)
call_command(display_hash)
call_command(sign_message)
call_command(display_signature)

$ openssl enc -rc4 -a -pbkdf2 -iter 1000000 -salt -in ./messages/plain/message4_from_alice.txt -out ./messages/encrypted/message4_from_alice.b64 -k /IcckmPr86b1ic9FdDw5sMe1ZKcaILyzubKh4yN1fBQ=

$ cat ./messages/encrypted/message4_from_alice.b64
U2FsdGVkX198rODpi3FhUeTI1MKzXq0wG9TDRsDhGDR9ji4s17hF4BSnmjFQIcRA
p1c=
$ openssl dgst -sha224 -out ./messages/hashed/message4_from_alice.hash ./messages/encrypted/message4_from_alice.b64
$ cat ./messages/hashed/message4_from_alice.hash
SHA224(./messages/encrypted/message4_from_alice.b64)= 7dbd0691b5f77d9d400f0b2693da6d39663e12ca2a7a99e0b7b56b50
$ openssl dgst -sha224 -sign ./DSA/alice_private_key.pem -out ./messages/signed/message4_from_alice.sign ./messages/hashed/message4_from_alice.hash
$ cat ./messages/signed/message4_from_alice.sign
0,K9 !���:�����d����O3S�[�*�"���<!NE��

Bob is now going to check the signature by getting the hash of the decrypted message, then comparing it with the signature, and decrypt the file
```
openssl dgst -sha224 -out ./messages/hashed/message4_from_alice.hash ./messages/encrypted/message4_from_alice.b64
openssl dgst -sha224 -verify ./DSA/alice_public_key.pem -signature ./messages/signed/message4_from_alice.sign ./messages/hashed/message4_from_alice.hash
openssl enc -rc4 -d -a -pbkdf2 -iter 1000000 -in ./messages/encrypted/message4_from_alice.b64 -out ./messages/decrypted/message4_from_alice.dec -k <symmetric_key>
```


In [29]:
#if files already exist, remove them
remove_files(["alice_message.dec", "hash_of_alice_enc_message.txt"])


hash_of_alice_message = "openssl dgst -sha224 -out ./messages/hashed/message4_from_alice.hash ./messages/encrypted/message4_from_alice.b64"
display_hash_of_alice_message = "cat ./messages/hashed/message4_from_alice.hash"
verify_signature = "openssl dgst -sha224 -verify ./DSA/alice_public_key.pem -signature ./messages/signed/message4_from_alice.sign ./messages/hashed/message4_from_alice.hash"
decrypt_message = f"openssl enc -rc4 -d -a -pbkdf2 -iter 1000000 -in ./messages/encrypted/message4_from_alice.b64 -out ./messages/decrypted/message4_from_alice.dec -k {key}"
display_decrypted_message = "cat ./messages/decrypted/message4_from_alice.dec"

call_command(hash_of_alice_message)
call_command(display_hash_of_alice_message)
call_command(verify_signature)
call_command(decrypt_message)
call_command(display_decrypted_message)


$ openssl dgst -sha224 -out ./messages/hashed/message4_from_alice.hash ./messages/encrypted/message4_from_alice.b64
$ cat ./messages/hashed/message4_from_alice.hash
SHA224(./messages/encrypted/message4_from_alice.b64)= 7dbd0691b5f77d9d400f0b2693da6d39663e12ca2a7a99e0b7b56b50
$ openssl dgst -sha224 -verify ./DSA/alice_public_key.pem -signature ./messages/signed/message4_from_alice.sign ./messages/hashed/message4_from_alice.hash
Verified OK
$ openssl enc -rc4 -d -a -pbkdf2 -iter 1000000 -in ./messages/encrypted/message4_from_alice.b64 -out ./messages/decrypted/message4_from_alice.dec -k /IcckmPr86b1ic9FdDw5sMe1ZKcaILyzubKh4yN1fBQ=

$ cat ./messages/decrypted/message4_from_alice.dec
This is a test message from Alice.

## 6. ChatGPT solution

I've asked chatGPT (version 3.5) to do the exercice.
Let's observe the answers.
Modifications on chatGPT's answers were made in the python code to change to input and output files, to make sure that the workspace stays organized.

### 6.1 Using RSA for exchanging a symmetric key
Here is the code generated by the IA.
```
# Generate RSA private key
openssl genpkey -algorithm RSA -out private_key.pem

# Extract RSA public key
openssl rsa -pubout -in private_key.pem -out public_key.pem

# Generate a symmetric key (e.g., AES-256)
openssl rand -base64 32 > symmetric_key.txt

# Encrypt the symmetric key with RSA public key
openssl rsautl -encrypt -inkey public_key.pem -pubin -in symmetric_key.txt -out encrypted_symmetric_key.txt

# Decrypt the symmetric key with RSA private key
openssl rsautl -decrypt -inkey private_key.pem -in encrypted_symmetric_key.txt -out decrypted_symmetric_key.txt
```
When compared with the code in chapter 1, the command are all the same. One of the only difference is the use of the parameter `ras_keygen_bits` on my code to specifically ask for 1024 instead of the default value 2048.  
It therefore seems that the IA solution work the same way as mine. 

In [30]:
#if files already exist, remove them
remove_files(["private_key.pem", "public_key.pem", "symmetric_key.txt", "encrypted_symmetric_key.txt"])


# Generate RSA private key
generate_rsa_key = "openssl genpkey -algorithm RSA -out ./RSA/GPT_private_key.pem"

# Extract RSA public key
extract_rsa_pub_key = "openssl rsa -pubout -in ./RSA/GPT_private_key.pem -out ./RSA/GPT_public_key.pem"

# Generate a symmetric key (e.g., AES-256)
gen_sym_key = "openssl rand -base64 32 > ./sym_keys/GPT_symmetric_key.txt"

# Encrypt the symmetric key with RSA public key
encrypt = "openssl rsautl -encrypt -inkey ./RSA/GPT_public_key.pem -pubin -in ./sym_keys/GPT_symmetric_key.txt -out ./sym_keys/GPT_encrypted_symmetric_key.txt"

# Decrypt the symmetric key with RSA private key
decrypt = "openssl rsautl -decrypt -inkey ./RSA/GPT_private_key.pem -in ./sym_keys/GPT_encrypted_symmetric_key.txt -out ./sym_keys/GPT_decrypted_symmetric_key.txt"


call_command(generate_rsa_key)
call_command(extract_rsa_pub_key)
#usual function do not work with ">", so we use subprocess
subprocess.run(["openssl", "rand", "-base64", "32"], stdout=open("./sym_keys/GPT_symmetric_key.txt", "x"))
call_command(encrypt)
call_command(decrypt)

$ openssl genpkey -algorithm RSA -out ./RSA/GPT_private_key.pem
$ openssl rsa -pubout -in ./RSA/GPT_private_key.pem -out ./RSA/GPT_public_key.pem
$ openssl rsautl -encrypt -inkey ./RSA/GPT_public_key.pem -pubin -in ./sym_keys/GPT_symmetric_key.txt -out ./sym_keys/GPT_encrypted_symmetric_key.txt
$ openssl rsautl -decrypt -inkey ./RSA/GPT_private_key.pem -in ./sym_keys/GPT_encrypted_symmetric_key.txt -out ./sym_keys/GPT_decrypted_symmetric_key.txt


..........+++++
..................................................+++++
writing RSA key


### 6.2 Encrypting and decrypting with RC4 in RFB mode:

Here is the code generated by the IA.
```
# Encrypt with RC4 in CFB mode using the symmetric key
openssl enc -rc4-cfb -in plaintext.txt -out encrypted_rc4.txt -pass file:decrypted_symmetric_key.txt

# Decrypt using RC4 in CFB mode
openssl enc -d -rc4-cfb -in encrypted_rc4.txt -out decrypted_rc4.txt -pass file:decrypted_symmetric_key.txt
```

Here, ChatGPT didn't do as well as the precedent chapter.  
He used the flag `-rc4-cfb` for the encryption method, which doesn't exist, so the command generated are not usable. But the flags `-rc2-cfb` and `-rc5-cfb` do exists, that could explain the mistake.

A difference between the answer at chapter 2 and chatGPT's answer is the usage of the two parameters `-iter` and `-pbkdf2`. These two parameters are optional, but not using them create a warning, as the old and default method of key derivation is deprecated, and the usage of these parameters makes OpenSSL use the new one.  
ChatGPT do not use them, so if the flag `-rc4-cfb` were to work, the command would still issue a warning.

In [31]:
#if files already exist, remove them
remove_files(["./messages/plain/GPT_plaintext.txt", "./messages/encrypted/GPT_encrypted.txt", "./messages/decrypted/GPT_decrypted.txt"])

with open("./messages/plain/GPT_plaintext.txt", "x") as f:
    f.write("This is a test message.")

encrypt = "openssl enc -rc4-cfb -in ./messages/plain/GPT_plaintext.txt -out ./messages/encrypted/GPT_encrypted_rc4.txt -pass file:./sym_keys/GPT_decrypted_symmetric_key.txt"
decrypt = "openssl enc -d -rc4-cfb -in ./messages/encrypted/GPT_encrypted_rc4.txt -out ./messages/decrypted/GPT_decrypted_rc4.txt -pass file:./sym_keys/GPT_decrypted_symmetric_key.txt"

call_command(encrypt)
call_command(decrypt)

$ openssl enc -rc4-cfb -in ./messages/plain/GPT_plaintext.txt -out ./messages/encrypted/GPT_encrypted_rc4.txt -pass file:./sym_keys/GPT_decrypted_symmetric_key.txt
Error:
Return code: 1
$ openssl enc -d -rc4-cfb -in ./messages/encrypted/GPT_encrypted_rc4.txt -out ./messages/decrypted/GPT_decrypted_rc4.txt -pass file:./sym_keys/GPT_decrypted_symmetric_key.txt
Error:
Return code: 1


enc: Unrecognized flag rc4-cfb
enc: Use -help for summary.
enc: Unrecognized flag rc4-cfb
enc: Use -help for summary.


### 6.3 Using SHA224 to hash a message
Here is the code generated by the IA
```
# Hash a message using SHA224
echo -n "YourMessageHere" | openssl dgst -sha224
```

This answer is quite different from the one at chapter 3, but it's mostly due to the fact that at the chapter three, the input and output are files. Here, we pass the message as a string in the command, and we write the hash in the terminal. But it still does what it's expected to do: Hash a string in SHA224.

In [32]:
hash = "echo -n \"YourMessageHere\" | openssl dgst -sha224"

subprocess.run(["echo", "-n", "\"YourMessageHere\""], stdout=subprocess.PIPE)
subprocess.run(["openssl", "dgst", "-sha224"], stdin=subprocess.PIPE)

(stdin)= d14a028c2a3a2bc9476102bb288234c415a2b01f828ea62ac5b3e42f


CompletedProcess(args=['openssl', 'dgst', '-sha224'], returncode=0)

### 6.4 Using DSA to sign a message

Here is the code generated by the IA
```
# Generate DSA parameters and private key
openssl dsaparam -out dsaparam.pem 2048
openssl gendsa -out private_dsa.pem dsaparam.pem

# Sign a message using DSA private key
echo -n "YourMessageHere" | openssl dgst -dss1 -sign private_dsa.pem -out signature.bin
```

Here, chatGPT used again a flag that doesn't exist, "-dss1", and he sign a non-hashed message, which can be done, but it's usually more efficent to sign the hash of the message.

In [33]:
#if files already exist, remove them
remove_files(["./DSA/GPT_param.pem", "./DSA/GPT_private_key.pem"])

create_param = "openssl dsaparam -out ./DSA/GPT_param.pem 2048"
gen_private_key = "openssl gendsa -out ./DSA/GPT_private_key.pem ./DSA/GPT_param.pem"

call_command(create_param)
call_command(gen_private_key)

subprocess.run(["echo", "-n", "\"YourMessageHere\""], stdout=subprocess.PIPE)
subprocess.run(["openssl", "dgst", "-dss1", "-sign", "./DSA/GPT_private_key.pem", "-out", "./messages/signed/GPT_signed_message1.bin"], stdin=subprocess.PIPE)

$ openssl dsaparam -out ./DSA/GPT_param.pem 2048


Generating DSA parameters, 2048 bit long prime
This could take some time
..............+.......+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++*
......+.....+...........+.......+......+.......................+............................+..+....................+...............+................+..+..+.+.+...+.............+.+...................+..+..+..........................+........+........+....+.........................+.......................................................+...........+..........+........................+.....+....+......+.....................+.........+....................................+.+..................................+..............+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++*
Generating DSA key, 2048 bits


$ openssl gendsa -out ./DSA/GPT_private_key.pem ./DSA/GPT_param.pem


dgst: Unrecognized flag dss1
dgst: Use -help for summary.


CompletedProcess(args=['openssl', 'dgst', '-dss1', '-sign', './DSA/GPT_private_key.pem', '-out', './messages/signed/GPT_signed_message1.bin'], returncode=1)

### 6.5 Signing and encrypting in different order
#### Signing and encrypting
```
# Sign the message using DSA private key
echo -n "YourMessageHere" | openssl dgst -dss1 -sign private_dsa.pem -out signed_message.bin

# Encrypt the signed message using RC4 in CFB mode with the symmetric key
openssl enc -rc4-cfb -in signed_message.bin -out encrypted_signed_message.txt -pass file:decrypted_symmetric_key.txt
```

#### Encrypting then signing
```
# Encrypt the message using RC4 in CFB mode with the symmetric key
openssl enc -rc4-cfb -in plaintext.txt -out encrypted_message.txt -pass file:decrypted_symmetric_key.txt

# Sign the encrypted message using DSA private key
openssl dgst -dss1 -sign private_dsa.pem -in encrypted_message.txt -out signed_encrypted_message.bin
```

In this answer, chatGPT used the same wrongs flags as the precedent answer.
He also encrypted the signature of the message instead of the message in the first part of the answer.

In [34]:

encrypt_signed_message = "openssl enc -rc4-cfb -in ./messages/signed/GPT_signed_message1.bin -out ./messages/encrypted/GPT_encrypted_signed_message1.bin -pass file:./sym_keys/GPT_decrypted_symmetric_key.txt"

encrypt_plain_message = "openssl enc -rc4-cfb -in ./messages/plain/GPT_plaintext.txt -out ./messages/encrypted/GPT_encrypted_message1.txt -pass file:./sym_keys/GPT_decrypted_symmetric_key.txt"
sign_encrypted_message = "openssl dgst -dss1 -sign ./DSA/GPT_private_key.pem -in ./messages/encrypted/GPT_encrypted_message1.txt -out ./messages/signed/GPT_signed_encrypted_message1.bin"

subprocess.run(["echo", "-n", "\"YourMessageHere\""], stdout=subprocess.PIPE)
subprocess.run(["openssl", "dgst", "-dss1", "-sign", "./DSA/GPT_private_key.pem", "-out", "./messages/signed/GPT_signed_message1.bin"], stdin=subprocess.PIPE)
call_command(encrypt_signed_message)

call_command(encrypt_plain_message)
call_command(sign_encrypted_message)

dgst: Unrecognized flag dss1
dgst: Use -help for summary.
enc: Unrecognized flag rc4-cfb
enc: Use -help for summary.


$ openssl enc -rc4-cfb -in ./messages/signed/GPT_signed_message1.bin -out ./messages/encrypted/GPT_encrypted_signed_message1.bin -pass file:./sym_keys/GPT_decrypted_symmetric_key.txt
Error:
Return code: 1
$ openssl enc -rc4-cfb -in ./messages/plain/GPT_plaintext.txt -out ./messages/encrypted/GPT_encrypted_message1.txt -pass file:./sym_keys/GPT_decrypted_symmetric_key.txt
Error:
Return code: 1
$ openssl dgst -dss1 -sign ./DSA/GPT_private_key.pem -in ./messages/encrypted/GPT_encrypted_message1.txt -out ./messages/signed/GPT_signed_encrypted_message1.bin
Error:
Return code: 1


enc: Unrecognized flag rc4-cfb
enc: Use -help for summary.
dgst: Unrecognized flag dss1
dgst: Use -help for summary.
